In [ ]:
# !pip install langchain-community langchain-openai langchain-chroma langchain langchain-huggingface 

In [1]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
# from langchain_huggingface.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_groq import ChatGroq
import os

/home/s.wendelken/.conda/envs/ehr-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "./data/"
loader = DirectoryLoader(path, glob="*.txt", loader_cls=TextLoader)
docs = loader.load()

In [3]:
from config import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY #smw_personal open ai"

In [5]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [6]:
from config import GOOGLE_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY # smw personal 

In [7]:
def generate_response(retriever, query):

    try:
        # llm = ChatGoogleGenerativeAI(model="gemini-pro")
        llm = ChatOpenAI(model="gpt-4")
        # llm =chat
#         llm = HuggingFacePipeline.from_model_id(
#     model_id="epfl-llm/meditron-7b",
#     task="text-generation",
# )
        template = """
            You are a helpful Medical AI assistant. Answer the question based on the context provided. Respond only if the answer is in the context.
            If the answer is not in the context then respond, Cannot answer the question.
            Stick to the context while answering and be concise.
            context: {context}
            question: {input}
            answer:
            """
        prompt = PromptTemplate.from_template(template)
        combine_docs_chain = create_stuff_documents_chain(llm, prompt)
        retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
        response = retrieval_chain.invoke({"input": query})
        # print(response["answer"])
        return response["answer"]
    except Exception as e:
        print(f"An error occurred while generating the response: {e}")
        return None

In [8]:
# questions = [
# "Given the History of Present Illness, What is the relevant past medical history?",
# "Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?",
# "Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?",
# "Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?",
# "given the history of present illness, What was the initial treatment course?",
# "Given the History of Present illness, What was the initial diagnostic work up and pertinent results?",
# "Given the radiology notes, What radiology, imaging or other studies that were performed and for what inducation? Summarize the impressions from each study. Make a table of Imaging study, indication, and impression."
# ]

# questions = ["Find the History of Present Illness Section.  Summarize why the patient was admitted to the hospital, the relevant past medical history, and the events in this section."]

# questions = ["Find the admission and discharge medication lists.  What medications were changed? What medications were new?"]

questions = ["Find and summarize the treatments that were administered to the patient"]

# questions = ["List the pertinent radiological and imaging results and summarize the impressions"]

In [9]:
response = generate_response(retriever, questions[0])
print(response)

The patient's treatment included a therapeutic and diagnostic paracentesis procedure where 3 L of clear orange fluid was removed from the right lower quadrant. The patient was also on several medications including: Valsartan 80 mg daily, Capecitabine 1500 mg daily, Humalog 10 units at breakfast and dinner, MetFORMIN (Glucophage) 500 mg daily, and Omeprazole 20 mg daily. The patient also had a nasogastric tube positioned appropriately.


In [20]:
response = generate_response(retriever, questions[0])
print(response)

The patient's past medical history includes chronic anemia, osteoporosis, hypertension, ataxia, recent L5 fracture due to recurrent falls, allergic rhinitis, irritable bowel syndrome, osteoarthritis, TMJ dysfunction, bilateral macular degeneration, glaucoma, cataract surgery, left shoulder hemiarthroplasty, bilateral salpingo-oophorectomy for benign ovarian mass, and umbilical hernia repair.


In [27]:
for question in questions:
    response = generate_response(retriever, question)
    print(question,"\n")
    print(response)
    print("\n*********************")

Given the History of Present Illness, What is the relevant past medical history? 

The patient's past medical history includes chronic anemia, osteoporosis, hypertension, ataxia, and a recent L5 fracture due to recurrent falls. Other past medical conditions are allergic rhinitis, irritable bowel syndrome, osteoarthritis, TMJ dysfunction, bilateral macular degeneration, glaucoma, cataract surgery, left shoulder hemiarthroplasty, bilateral salpino-oophorectomy for benign ovarian mass, and umbilical hernia repair.

*********************
Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital? 

The patient was admitted to the hospital due to weakness and diarrhea. The patient has a history of chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture. The weakness and diarrhea were likely due to her chronic anemia 

In [28]:
retriever?

Type:           VectorStoreRetriever
String form:    tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x2ab2d48dded0> search_kwargs={'k': 1}
File:           ~/.conda/envs/ehr-env/lib/python3.11/site-packages/langchain_core/vectorstores.py
Docstring:      Base Retriever class for VectorStore.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.